## Homework 3 - Disha Gandhi

In [1]:
import numpy as np
import gurobipy as gp

#### Question 2

A company is thinking about building new facilities in Austin and Dallas. Here is the relevant data.
Total capital available for investment is 11M dollars. You can’t build more than one factory (warehouse) in one place. At most one warehouse must be built in Austin or Dallas. At least one factory must be built in Austin or Dallas. Find the optimal investment strategy.
Is building the Factory in Dallas part of the optimal investment strategy?

In [15]:
obj = np.array([9,5,6,4])
A = np.zeros((3,4))
A[0,:] = [6,3,5,2]
A[1,:] = [0,0,1,1]
A[2,:] = [1,1,0,0]
sense = ['<','<','>']
b = np.array([11,1,1])

In [16]:
ipMod = gp.Model()
ipMod_x = ipMod.addMVar(4,vtype=['I','I','I','I'],ub=np.array([1,1,1,1])) # 'C' or 'I' or 'B'
ipMod_con = ipMod.addMConstrs(A, ipMod_x, sense, b)
ipMod.setMObjective(None,obj,0,sense=gp.GRB.MAXIMIZE)

ipMod.Params.OutputFlag = 0 # tell gurobi to shut up!!
ipMod.Params.TimeLimit = 3600
ipMod.optimize()

In [17]:
ipMod.objVal # optimal revenue level

18.0

In [18]:
ipMod.x

[1.0, 1.0, -0.0, 1.0]

#### Question 3

Western Airlines wants to design a hub system in the United States. Each hub is used for connecting flights to and from cities within 1000 miles of the hub. Western runs flights among the following cities: Atlanta (ATL), Boston (BOS), Chicago (CHI), Denver (DEN), Houston (HOU), Los Angeles (LAX), New Orleans (NO), New York (NY), Pittsburgh (PIT), Salt Lake City (SLC), San Francisco (SF), and Seattle (SEA). The company wants to determine the smallest number of hubs it needs to cover all these cities, where a city is covered if it is within 1000 miles of at least one hub. The table below lists which cities are within 1000 miles of other cities. For example, if a hub was placed at Boston (BOS), it could cover the cities of Boston, New York, and Pittsburgh.
Formulate and solve this problem as a binary integer program.

Is SLC a hub in the optimal solution?

In [2]:
A = np.zeros([12,12])
A[0:] =[1,0,1,0,1,0,1,1,1,0,0,0]
A[1:] =[0,1,0,0,0,0,0,1,1,0,0,0]
A[2:] =[1,0,1,0,0,0,1,1,1,0,0,0]
A[3:] =[0,0,0,1,0,0,0,0,0,1,0,0]
A[4:] =[1,0,0,0,1,0,1,0,0,0,0,0]
A[5:] =[0,0,0,0,0,1,0,0,0,1,1,0]
A[6:] =[1,0,1,0,1,0,1,0,0,0,0,0]
A[7:] =[1,1,1,0,0,0,0,1,1,0,0,0]
A[8:] =[1,1,1,0,0,0,0,1,1,0,0,0]
A[9:] =[0,0,0,1,0,1,0,0,0,1,1,1]
A[10:] =[0,0,0,0,0,1,0,0,0,1,1,1]
A[11:] =[0,0,0,0,0,0,0,0,0,1,1,1]

obj=np.array([1,1,1,1,1,1,1,1,1,1,1,1])

sense = (np.array(['>']*12))

b = np.array([1]*12)


ub=np.array([1]*12)
lb=np.array([0]*12)

In [3]:
npvMod = gp.Model()
npvMod_x = npvMod.addMVar(len(obj),vtype=['B']*len(obj)) # 'C' or 'I' or 'B'
npvMod_con = npvMod.addMConstrs(A, npvMod_x, sense, b)
npvMod.setMObjective(None,obj,0,sense=gp.GRB.MINIMIZE)

npvMod.Params.OutputFlag = 0 # tell gurobi to shut up!!
npvMod.optimize()

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-11


In [4]:
npvMod_x.x

array([1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0.])

In [5]:
npvMod.objVal

3.0

#### Question 4

A paper mill cuts the rolls of paper into different widths to satisfy customers’ demand. In this problem, assume the original rolls of paper are 120 inches wide. The table below shows the orders received by the paper mill.
A 120 inch roll can be cut in many ways. For example, we can cut four 25-inch rolls while wasting the remaining 20 inches; we can also cut one 25-inch, one 37-inch, and one 54-inch. In the second case, only 4 inches is wasted.
Develop and solve an integer program to minimize the waste from cutting to satisfy all orders. For more information, you can check out the cutting stock problem on wikipedia:

How many rolls get cut into the pattern such that there are 3 cuts of 25 inches, and 1 cut of 37 inches (this pattern results in 8" waste)?

In [6]:
combo_list = []
for i in range(5):# for x
    for j in range(4):#for y
        for k in range(3):
            if 25*i+37*j+54*k <= 120 and (120 - (25*i+37*j+54*k))<25:
                temp = (i,j,k)
                combo_list.append(temp)
print(combo_list)

[(0, 0, 2), (0, 3, 0), (1, 1, 1), (1, 2, 0), (2, 0, 1), (3, 1, 0), (4, 0, 0)]


In [8]:
obj = np.zeros(len(combo_list))
A = np.zeros((3,len(combo_list)))
b = np.array([233,148,106])
sense = np.array(['>']*3)
vtype = np.array(['I']*len(combo_list))

for i in range(len(combo_list)):
    obj[i] = 120 - (25*combo_list[i][0]+37*combo_list[i][1]+54*combo_list[i][2])
    A[0,i] = combo_list[i][0]
    A[1,i] = combo_list[i][1]
    A[2,i] = combo_list[i][2]


In [10]:
ipMod_4 = gp.Model()
ipMod_x4 = ipMod_4.addMVar(len(combo_list),vtype=vtype) # 'C' or 'I' or 'B', C = continuous variables, I = Integer variables &
# B = Binary variables (1 or 0)
ipMod_con4 = ipMod_4.addMConstrs(A, ipMod_x4, sense, b)
ipMod_4.setMObjective(None,obj,0)##sense=gp.GRB.MAXIMIZE)
ipMod_4.Params.OutputFlag = 0 # tell gurobi to shut up!!
ipMod_4.optimize()

In [11]:
ipMod_x4.x

array([ -0.,  -0., 107.,  -0.,  -0.,  42.,  -0.])

In [12]:
ipMod_4.ObjVal

764.0

#### Question 5

The days-off scheduling problem must be solved routinely by businesses that operate 6 or 7 days a week. Examples include hospitals, airlines, municipal transportation companies, and the postal service. The most common example is the (5,7)-cyclic staffing problem. The objective of it is to minimize the cost of assigning workers to a 7-day cyclic schedule so that
1) Sufficient workers are available every day.
2) Each person works 5 consecutive days and is idle to the remaining 2 days.
Here is the table showing the cost of having an employee for each day and the number of employees required for each day.
For example, the pattern that one works from Sunday to Thursday costs 90 + 60 × 4 = 330. Formulate and solve an integer programming problem to represent this problem.
How many employees work Monday-Friday?

In [30]:
obj = np.array([300,330,360,360,360,360,330])
A = np.zeros((7,7))
A[0,:] = [0,0,1,1,1,1,1]
A[1,:] = [1,0,0,1,1,1,1]
A[2,:] = [1,1,0,0,1,1,1]
A[3,:] = [1,1,1,0,0,1,1]
A[4,:] = [1,1,1,1,0,0,1]
A[5,:] = [1,1,1,1,1,0,0]
A[6,:] = [0,1,1,1,1,1,0]
sense = np.array(['>','>','>','>','>','>','>'])
b = np.array([5,13,12,10,14,8,6])

In [33]:
ipMod = gp.Model()
ipMod_x = ipMod.addMVar(7,vtype=['I','I','I','I','I','I','I']) # 'C' or 'I' or 'B'
ipMod_con = ipMod.addMConstrs(A, ipMod_x, sense, b)
ipMod.setMObjective(None,obj,0,sense=gp.GRB.MINIMIZE)

ipMod.Params.OutputFlag = 0 # tell gurobi to shut up!!
ipMod.Params.TimeLimit = 3600
ipMod.optimize()

In [34]:
ipMod.objVal # optimal revenue level

4830.0

In [35]:
ipMod.x

[8.0, 2.0, -0.0, 3.0, 1.0, -0.0, 1.0]